In [1]:
%load_ext autoreload
%autoreload 2

## Testing the Composites Discovery App with n=3 phases

In [2]:
from hashin_shtrikman_mp.core.user_input import MaterialProperty, Material, MixtureProperty, Mixture, UserInput

# Define properties for each material
properties_mat_1 = [
    MaterialProperty(prop='elec_cond_300k_low_doping', upper_bound=120, lower_bound=1e-7), # upper_bound=20, lower_bound=1 
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2, lower_bound=1e-7), # upper_bound=0.0001, lower_bound=1e-5 
    MaterialProperty(prop='bulk_modulus', upper_bound=500, lower_bound=50),
    MaterialProperty(prop='shear_modulus', upper_bound=500, lower_bound=80),
    MaterialProperty(prop='universal_anisotropy', upper_bound=6, lower_bound=1),
]

properties_mat_2 = [
    MaterialProperty(prop='elec_cond_300k_low_doping', upper_bound=78, lower_bound=1e-7), # upper_bound=5, lower_bound=2
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2, lower_bound=1e-7), # upper_bound=0.009, lower_bound=1e-4
    MaterialProperty(prop='bulk_modulus', upper_bound=400, lower_bound=20),
    MaterialProperty(prop='shear_modulus', upper_bound=500, lower_bound=100),
    MaterialProperty(prop='universal_anisotropy', upper_bound=4.3, lower_bound=1.3),
]

properties_mat_3 = [
    MaterialProperty(prop='elec_cond_300k_low_doping', upper_bound=78, lower_bound=1e-7), # upper_bound=10, lower_bound=1
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2, lower_bound=1e-7), # upper_bound=0.005, lower_bound=1e-4
    MaterialProperty(prop='bulk_modulus', upper_bound=700, lower_bound=20),
    MaterialProperty(prop='shear_modulus', upper_bound=600, lower_bound=100),
    MaterialProperty(prop='universal_anisotropy', upper_bound=2.1, lower_bound=0.9),
]

# Define properties for the mixture
properties_mixture = [
    MixtureProperty(prop='elec_cond_300k_low_doping', desired_prop=9),
    MixtureProperty(prop='therm_cond_300k_low_doping', desired_prop=0.9),
    MixtureProperty(prop='bulk_modulus', desired_prop=280),
    MixtureProperty(prop='shear_modulus', desired_prop=230),
    MixtureProperty(prop='universal_anisotropy', desired_prop=1.5),
]

# Create Material & Mixture instances
mat_1 = Material(name='mat_1', properties=properties_mat_1)
mat_2 = Material(name='mat_2', properties=properties_mat_2)
mat_3 = Material(name='mat_3', properties=properties_mat_3)
mixture = Mixture(name='mixture', properties=properties_mixture)
aggregate = [mat_1, mat_2, mat_3, mixture]

# Initialize UserInput instance with materials and mixtures
user_input= UserInput(materials=[mat_1, mat_2, mat_3], mixtures=[mixture])
print("User Input: ", user_input)

# Initialize dictionaries to store the overall upper and lower bounds for each property
overall_bounds = {}

# Iterate over materials
for entity in aggregate:
    # Skip the mixture as it doesn't have upper and lower bounds
    if isinstance(entity, Material):
        for property in entity.properties:
            prop_name = property.prop

            # Initialize the overall bounds if they are not already present for the property
            if prop_name not in overall_bounds:
                overall_bounds[prop_name] = {'upper_bound': property.upper_bound, 'lower_bound': property.lower_bound}
            else:
                # Update overall upper and lower bounds by comparing with existing values
                overall_bounds[prop_name]['upper_bound'] = max(overall_bounds[prop_name]['upper_bound'], property.upper_bound)
                overall_bounds[prop_name]['lower_bound'] = min(overall_bounds[prop_name]['lower_bound'], property.lower_bound)

# Print the overall bounds for each property
print("Overall Upper & Lower Bounds:")
for prop, bounds in overall_bounds.items():
    print(f"Property: {prop}, Upper Bound: {bounds['upper_bound']}, Lower Bound: {bounds['lower_bound']}")

# Step 1: Create the consolidated_dict
overall_bounds_dict = {}
for prop, bounds in overall_bounds.items():
    overall_bounds_dict[prop] = {
        'upper_bound': bounds['upper_bound'],
        'lower_bound': bounds['lower_bound']
    }

print(overall_bounds_dict)

/Users/carlabecker/opt/anaconda3/envs/carla_mp_api/lib/python3.9/site-packages/mpcontribs/client/__init__.py:88: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.set_option('mode.use_inf_as_na', True)


User Input:  {'mat_1': {'elec_cond_300k_low_doping': {'upper_bound': 120.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 500.0, 'lower_bound': 50.0}, 'shear_modulus': {'upper_bound': 500.0, 'lower_bound': 80.0}, 'universal_anisotropy': {'upper_bound': 6.0, 'lower_bound': 1.0}}, 'mat_2': {'elec_cond_300k_low_doping': {'upper_bound': 78.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 400.0, 'lower_bound': 20.0}, 'shear_modulus': {'upper_bound': 500.0, 'lower_bound': 100.0}, 'universal_anisotropy': {'upper_bound': 4.3, 'lower_bound': 1.3}}, 'mat_3': {'elec_cond_300k_low_doping': {'upper_bound': 78.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 700.0, 'lower_bound': 20.0}, 'shear_modulus': {'upper_bound': 600.0, 'lower_bound': 100.0}, 'unive

In [3]:
from hashin_shtrikman_mp.core.optimizer import Optimizer
optimizer = Optimizer(api_key="uJpFxJJGKCSp9s1shwg9HmDuNjCDfWbM", user_input=user_input)

print("Property categories: ", optimizer.property_categories)
print("Property docs: ", optimizer.property_docs)

print("Lower Bounds:", optimizer.lower_bounds)
print("Upper Bounds:", optimizer.upper_bounds)

print("Number of Materials:", optimizer.num_materials)
print("Number of Properties:", optimizer.num_properties)

2024-11-25 21:39:32,187 - hashin_shtrikman_mp.log.custom_logger - INFO - Loading property categories from /Users/carlabecker/Library/Mobile Documents/com~apple~CloudDocs/Carla's Desktop/UC Berkeley/Research/Materials Project/hashin_shtrikman_mp/src/hashin_shtrikman_mp/core/../io/inputs/mp_property_docs.yaml.
2024-11-25 21:39:32,193 - hashin_shtrikman_mp.log.custom_logger - INFO - property_categories = ['carrier-transport', 'elastic']
2024-11-25 21:39:32,194 - hashin_shtrikman_mp.log.custom_logger - INFO - mixture_props = {'elec_cond_300k_low_doping': {'desired_prop': 9.0}, 'therm_cond_300k_low_doping': {'desired_prop': 0.9}, 'bulk_modulus': {'desired_prop': 280.0}, 'shear_modulus': {'desired_prop': 230.0}, 'universal_anisotropy': {'desired_prop': 1.5}}


self.nuild_dict: <bound method UserInput.build_dict of {'mat_1': {'elec_cond_300k_low_doping': {'upper_bound': 120.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 500.0, 'lower_bound': 50.0}, 'shear_modulus': {'upper_bound': 500.0, 'lower_bound': 80.0}, 'universal_anisotropy': {'upper_bound': 6.0, 'lower_bound': 1.0}}, 'mat_2': {'elec_cond_300k_low_doping': {'upper_bound': 78.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 400.0, 'lower_bound': 20.0}, 'shear_modulus': {'upper_bound': 500.0, 'lower_bound': 100.0}, 'universal_anisotropy': {'upper_bound': 4.3, 'lower_bound': 1.3}}, 'mat_3': {'elec_cond_300k_low_doping': {'upper_bound': 78.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 700.0, 'lower_bound': 20.0}, 'shear_modulus': {'upper_bo

In [4]:
# # Testing without calls to generate final dict (faster)
# import json
# consolidated_dict = {}
# with open("consolidated_dict_02_11_2024_23_45_58") as f:
#     consolidated_dict = json.load(f)
# print(consolidated_dict)

In [5]:
# Run the optimization to obtain n=3 materials and their volume fractions that achieve the desired composite material properties
optimizer.set_HS_optim_params(gen_counter=True)

Generation 0 of 100
Generation 1 of 100
Generation 2 of 100
Generation 3 of 100
Generation 4 of 100
Generation 5 of 100
Generation 6 of 100
Generation 7 of 100
Generation 8 of 100
Generation 9 of 100
Generation 10 of 100
Generation 11 of 100
Generation 12 of 100
Generation 13 of 100
Generation 14 of 100
Generation 15 of 100
Generation 16 of 100
Generation 17 of 100
Generation 18 of 100
Generation 19 of 100
Generation 20 of 100
Generation 21 of 100
Generation 22 of 100
Generation 23 of 100
Generation 24 of 100
Generation 25 of 100
Generation 26 of 100
Generation 27 of 100
Generation 28 of 100
Generation 29 of 100
Generation 30 of 100
Generation 31 of 100
Generation 32 of 100
Generation 33 of 100
Generation 34 of 100
Generation 35 of 100
Generation 36 of 100
Generation 37 of 100
Generation 38 of 100
Generation 39 of 100
Generation 40 of 100
Generation 41 of 100
Generation 42 of 100
Generation 43 of 100
Generation 44 of 100
Generation 45 of 100
Generation 46 of 100
Generation 47 of 100
Ge

In [6]:
# Create an instance of the Visualizer class using the instance of the Optimizer class
from hashin_shtrikman_mp.core.visualizer import Visualizer
visualizer = Visualizer(optimizer)

2024-11-25 21:39:37,437 - hashin_shtrikman_mp.log.custom_logger - INFO - Loading property categories from /Users/carlabecker/Library/Mobile Documents/com~apple~CloudDocs/Carla's Desktop/UC Berkeley/Research/Materials Project/hashin_shtrikman_mp/src/hashin_shtrikman_mp/core/../io/inputs/mp_property_docs.yaml.
2024-11-25 21:39:37,443 - hashin_shtrikman_mp.log.custom_logger - INFO - property_categories = ['carrier-transport', 'elastic']
2024-11-25 21:39:37,443 - hashin_shtrikman_mp.log.custom_logger - INFO - mixture_props = {'elec_cond_300k_low_doping': {'desired_prop': 9.0}, 'therm_cond_300k_low_doping': {'desired_prop': 0.9}, 'bulk_modulus': {'desired_prop': 280.0}, 'shear_modulus': {'desired_prop': 230.0}, 'universal_anisotropy': {'desired_prop': 1.5}}


self.nuild_dict: <bound method UserInput.build_dict of {'mat_1': {'elec_cond_300k_low_doping': {'upper_bound': 120.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 500.0, 'lower_bound': 50.0}, 'shear_modulus': {'upper_bound': 500.0, 'lower_bound': 80.0}, 'universal_anisotropy': {'upper_bound': 6.0, 'lower_bound': 1.0}}, 'mat_2': {'elec_cond_300k_low_doping': {'upper_bound': 78.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 400.0, 'lower_bound': 20.0}, 'shear_modulus': {'upper_bound': 500.0, 'lower_bound': 100.0}, 'universal_anisotropy': {'upper_bound': 4.3, 'lower_bound': 1.3}}, 'mat_3': {'elec_cond_300k_low_doping': {'upper_bound': 78.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 700.0, 'lower_bound': 20.0}, 'shear_modulus': {'upper_bo

In [7]:
# Print the optimization results as a table
visualizer.print_table_of_best_designs(rows=10)

In [8]:
# Plot the genetic algorithm convergence plot
visualizer.plot_optimization_results()

In [17]:
# Plot the contributions to the cost function for the best performer
visualizer.plot_cost_func_contribs()

Cost of best design: 0.0642334422680568


In [10]:
# Create an instance of the MatchFinder class using the instance of the Optimizer class
from hashin_shtrikman_mp.core.match_finder import MatchFinder
match_finder = MatchFinder(optimizer)

2024-11-25 21:39:38,088 - hashin_shtrikman_mp.log.custom_logger - INFO - Loading property categories from /Users/carlabecker/Library/Mobile Documents/com~apple~CloudDocs/Carla's Desktop/UC Berkeley/Research/Materials Project/hashin_shtrikman_mp/src/hashin_shtrikman_mp/core/../io/inputs/mp_property_docs.yaml.
2024-11-25 21:39:38,095 - hashin_shtrikman_mp.log.custom_logger - INFO - property_categories = ['carrier-transport', 'elastic']
2024-11-25 21:39:38,095 - hashin_shtrikman_mp.log.custom_logger - INFO - mixture_props = {'elec_cond_300k_low_doping': {'desired_prop': 9.0}, 'therm_cond_300k_low_doping': {'desired_prop': 0.9}, 'bulk_modulus': {'desired_prop': 280.0}, 'shear_modulus': {'desired_prop': 230.0}, 'universal_anisotropy': {'desired_prop': 1.5}}


self.nuild_dict: <bound method UserInput.build_dict of {'mat_1': {'elec_cond_300k_low_doping': {'upper_bound': 120.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 500.0, 'lower_bound': 50.0}, 'shear_modulus': {'upper_bound': 500.0, 'lower_bound': 80.0}, 'universal_anisotropy': {'upper_bound': 6.0, 'lower_bound': 1.0}}, 'mat_2': {'elec_cond_300k_low_doping': {'upper_bound': 78.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 400.0, 'lower_bound': 20.0}, 'shear_modulus': {'upper_bound': 500.0, 'lower_bound': 100.0}, 'universal_anisotropy': {'upper_bound': 4.3, 'lower_bound': 1.3}}, 'mat_3': {'elec_cond_300k_low_doping': {'upper_bound': 78.0, 'lower_bound': 1e-07}, 'therm_cond_300k_low_doping': {'upper_bound': 2.0, 'lower_bound': 1e-07}, 'bulk_modulus': {'upper_bound': 700.0, 'lower_bound': 20.0}, 'shear_modulus': {'upper_bo

In [13]:
# Get material matches
matches_dict = match_finder.get_material_matches(overall_bounds_dict, user_input)
print(f'Material Matches: {matches_dict}')

Retrieving SummaryDoc documents:   0%|          | 0/256 [00:00<?, ?it/s]

Initialized result_dict: {'material_id': [], 'formula_pretty': [], 'elec_cond_300k_low_doping': [], 'therm_cond_300k_low_doping': [], 'bulk_modulus_vrh': [], 'shear_modulus_vrh': [], 'universal_anisotropy': []}
Carrier transport is in the property categories
Material Matches: {}


In [14]:
import os
import json
from datetime import datetime

# Directory where the files are located
directory = './'

# Function to get the latest consolidated dict file
def get_latest_consolidated_dict_file(directory):
    # List all files in the directory
    files = os.listdir(directory)
    
    # Filter out only the consolidated_dict files
    consolidated_files = [f for f in files if f.startswith('consolidated_dict_')]
    
    # Sort the files by date and time in the filename
    consolidated_files.sort(key=lambda x: datetime.strptime(x, 'consolidated_dict_%m_%d_%Y_%H_%M_%S'))
    
    # Return the latest file
    latest_file = consolidated_files[-1] if consolidated_files else None
    return latest_file

# Get the latest consolidated dict file
latest_file = get_latest_consolidated_dict_file(directory)

if latest_file:
    # Open the latest consolidated dict file
    with open(os.path.join(directory, latest_file), 'r') as f:
        consolidated_dict = json.load(f)
        print(f"Loaded consolidated_dict from {latest_file}")
else:
    print("No consolidated dict file found.")


match_finder.get_material_match_costs(matches_dict=matches_dict, consolidated_dict=consolidated_dict)

Loaded consolidated_dict from consolidated_dict_11_25_2024_21_39_58
No materials match the recommended composite formulation.
